In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming, solve_tsp_branch_and_bound

In [2]:
# Генерация случайной матрицы расстояний
def generate_random_distance_matrix(num_cities, seed=None):
    if seed is not None:
        np.random.seed(seed)
    matrix = np.random.rand(num_cities, num_cities)
    matrix = (matrix + matrix.T) / 2  # Симметричная матрица
    np.fill_diagonal(matrix, 0)  # Расстояние до самого себя равно 0
    return matrix

# Вычисление полной длины маршрута
def calculate_total_distance(route, distance_matrix):
    total_distance = 0
    for i in range(len(route)):
        total_distance += distance_matrix[route[i], route[(i + 1) % len(route)]]
    return total_distance

In [3]:
# Encoder для Pointer Network
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden  # Возвращаем только скрытое состояние

# Decoder для Pointer Network
class Decoder(nn.Module):
    def __init__(self, hidden_dim, input_dim):
        super(Decoder, self).__init__()
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.attention = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        self.input_transform = nn.Linear(input_dim, hidden_dim)  # Преобразование входных данных

    def forward(self, prev_hidden, encoder_outputs):
        # Преобразуем входные данные к размерности hidden_dim
        lstm_input = self.input_transform(prev_hidden.unsqueeze(1))
        lstm_out, hidden = self.lstm(lstm_input)
        combined = torch.cat([encoder_outputs, lstm_out.expand(-1, encoder_outputs.size(1), -1)], dim=2)
        attention_scores = self.attention(combined).squeeze(2)
        return attention_scores, hidden


In [4]:
# Pointer Network
class PointerNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(PointerNetwork, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim)
        self.decoder = Decoder(hidden_dim, input_dim)

    def forward(self, distance_matrix, target_sequence=None, teacher_forcing_ratio=0.5):
        batch_size, num_cities, _ = distance_matrix.size()
        encoder_outputs, hidden = self.encoder(distance_matrix)
        decoder_input = torch.zeros(batch_size, num_cities, device=distance_matrix.device)
        decoder_hidden = hidden.squeeze(0)

        attentions = []

        for i in range(num_cities):
            attention_scores, decoder_hidden = self.decoder(decoder_input, encoder_outputs)
            attentions.append(attention_scores)

            # Преобразуем вероятности в индексы для следующего шага
            probabilities = torch.softmax(attention_scores, dim=1)
            _, index = torch.max(probabilities, dim=1)

            if target_sequence is not None and np.random.rand() < teacher_forcing_ratio:
                decoder_input = distance_matrix[torch.arange(batch_size), target_sequence[:, i]]
            else:
                decoder_input = distance_matrix[torch.arange(batch_size), index]

        return torch.stack(attentions, dim=1)  # Возвращаем все attention scores


In [5]:
# Обучение модели
def train_model(model, optimizer, distance_matrix, target_sequence, criterion):
    model.train()
    optimizer.zero_grad()
    
    # Получаем предсказания (все attention scores)
    predicted_scores = model(distance_matrix, target_sequence)
    
    # Перестраиваем предсказания и метки для CrossEntropyLoss
    predicted_scores = predicted_scores.view(-1, predicted_scores.size(-1))  # (N, C)
    target_sequence_flat = target_sequence.view(-1)  # (N,)
    
    loss = criterion(predicted_scores.float(), target_sequence_flat.long())
    loss.backward()
    optimizer.step()
    return loss.item()


In [6]:
# Параметры
num_cities = 12
hidden_dim = 128
batch_size = 32
epochs = 1500
learning_rate = 0.0002

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
# Генерация данных
#distance_matrix = generate_random_distance_matrix(num_cities)
#distance_matrix_tensor = torch.tensor(distance_matrix, dtype=torch.float32).unsqueeze(0).repeat(batch_size, 1, 1)
distance_matrix_tensor = torch.tensor(np.load('X_train.npy'), dtype=torch.float32)
distance_matrix_tensor = distance_matrix_tensor.to(device)

In [9]:
# Целевая последовательность (для обучения)
#target_sequence = torch.randperm(num_cities).unsqueeze(0).repeat(batch_size, 1)
target_sequence = torch.tensor(np.load('Y_train.npy'), dtype=torch.int)
target_sequence = target_sequence.to(device)

In [10]:
# Инициализация модели
model = PointerNetwork(input_dim=num_cities, hidden_dim=hidden_dim)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [11]:
next(model.parameters()).is_cuda

True

In [12]:
# Обучение
for epoch in range(epochs):
    loss = train_model(model, optimizer, distance_matrix_tensor, target_sequence, criterion)
    if epoch % 25 == 0 or epoch == epoch - 1:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

/tmp/ipykernel_2627/3108773192.py:16: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/aten/src/ATen/native/Scalar.cpp:22.)
  return loss.item()


Epoch 0, Loss: 2.4856
Epoch 25, Loss: 2.4719
Epoch 50, Loss: 2.3804
Epoch 75, Loss: 2.2467
Epoch 100, Loss: 2.2074
Epoch 125, Loss: 2.1984
Epoch 150, Loss: 2.1929
Epoch 175, Loss: 2.1936
Epoch 200, Loss: 2.1680
Epoch 225, Loss: 2.1575
Epoch 250, Loss: 2.1694
Epoch 275, Loss: 2.1744
Epoch 300, Loss: 2.1472
Epoch 325, Loss: 2.1837
Epoch 350, Loss: 2.0963
Epoch 375, Loss: 2.1786
Epoch 400, Loss: 2.0997
Epoch 425, Loss: 2.1103
Epoch 450, Loss: 2.1066
Epoch 475, Loss: 2.0948
Epoch 500, Loss: 2.0440
Epoch 525, Loss: 2.0861
Epoch 550, Loss: 2.0491
Epoch 575, Loss: 2.1250
Epoch 600, Loss: 2.0349
Epoch 625, Loss: 2.0426
Epoch 650, Loss: 2.0048
Epoch 675, Loss: 2.0808
Epoch 700, Loss: 2.0180
Epoch 725, Loss: 2.0077
Epoch 750, Loss: 2.0882
Epoch 775, Loss: 1.9153
Epoch 800, Loss: 1.9359
Epoch 825, Loss: 1.9863
Epoch 850, Loss: 2.1223
Epoch 875, Loss: 2.0533
Epoch 900, Loss: 2.0018
Epoch 925, Loss: 1.9265
Epoch 950, Loss: 1.8917
Epoch 975, Loss: 1.8665
Epoch 1000, Loss: 1.9331
Epoch 1025, Loss: 1.

In [17]:
# torch.save(model, 'my_t_model')

In [43]:
# Model class must be defined somewhere
# model = torch.load('my_t_model', weights_only=False)
model.eval()

PointerNetwork(
  (encoder): Encoder(
    (lstm): LSTM(12, 128, batch_first=True)
  )
  (decoder): Decoder(
    (lstm): LSTM(128, 128, batch_first=True)
    (attention): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=1, bias=True)
    )
    (input_transform): Linear(in_features=12, out_features=128, bias=True)
  )
)

In [38]:
print("Total Distance:", calculate_total_distance(target_sequence[0], distance_matrix_tensor[0]))

Total Distance: tensor(167.8861, device='cuda:0')


In [41]:
def predict_route(a):
    with torch.no_grad():
        predicted_score = model(a.unsqueeze(0))[0]
        route = []
        dist = 0
        current = -1
        for r in predicted_score:
            r[route] = -np.inf
            next_n = torch.argmax(r)
            route.append(int(next_n))
            if current > 0:
                dist += a[current, next_n]
            current = next_n
        dist += a[current, route[0]]
        return route, dist

In [42]:
predict_route(distance_matrix_tensor[0])

([0, 5, 11, 9, 10, 8, 4, 3, 7, 6, 2, 1], tensor(223.5871, device='cuda:0'))

In [ ]:
cnt = 50
ld = []
lp = []
lg = []
lr = []
for i in tqdm(range(cnt)):
    a = rand.normal(20, 5, size=(N, N))
    # a = np.round((a + a.T)/2,2)
    a[np.eye(N)==1] = 0
    permutation, distance = solve_tsp_dynamic_programming(a)
    route, total_dist = predict_route(solver, a, False, 500)
    route, dist = predict_geedy_route(a)
    route, rdist = predict_route(solver, a, True, 500)
    ld.append(distance)
    lp.append(total_dist)
    lg.append(dist)
    lr.append(rdist)
Y_predict = np.array(lp)
Y_true = np.array(ld)
Y_greedy = np.array(lg)
Y_rnd = np.array(lr)